In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("history.csv")

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_5602/3951251560.py:1: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("history.csv")


In [3]:
df = df.iloc[:, -27:]
df.head()

,is_broker_account,is_inventory_account,is_gl_account,is_control_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,...,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,f,f,f,f,t,2021-12-16 00:00:00,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,f,f,f,f,t,2023-01-03 00:00:00,BATCH,f,NaN,0000,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,f,f,f,f,t,2021-03-15 00:00:00,GUERINO,f,NaN,00/0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn
3,f,f,f,f,t,2021-01-04 00:00:00,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,f,f,f,f,t,2023-06-27 00:00:00,H01,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,NaN,NaN,f,Churn


In [4]:
def drop_cols(df, cols):
    df = df.drop(cols, axis=1)
    return df

In [5]:
def boolean_map(df, cols):
    for col in cols:
        df[col] = df[col].fillna('f').map({'t': 1, 'f': 0})
    return df

In [6]:
# Commentary: Standardized value from 0 to 1. Filled blanks with 0.
def parse_time(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
        min_date = df[col].min()
        df[col] = df[col].fillna(min_date)

        df[col] = df[col].astype(int)
        min = df[col].min()
        max = df[col].max()    
        df[col] = (df[col] - min) / (max - min)

    return df

In [7]:
# 1. Drop Useless
drop_columns = ['is_broker_account', 'is_gl_account', 'is_control_account', 'is_plan_grandfathered']
df = drop_cols(df, drop_columns)

In [8]:
# 2. Map Booleans
bool_columns = ['is_inventory_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'is_family_resp', 'is_hrdc_resp']
df = boolean_map(df, bool_columns)

bool_maybe_columns = ['is_olob', 'visible_in_reports', 'use_original_date_for_payment_calc']
df = boolean_map(df, bool_maybe_columns)

In [9]:
# 3. Parse Times
time_columns = ['last_maintenance_time', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']

df = parse_time(df, time_columns)

In [10]:
# 4. Miscellaneous/Unsure
df.number_of_beneficiaries.fillna(0, inplace=True)
df.label = df.label.map({'No Churn': 0, 'Churn': 1})

Looking for guidance on these columns
1. last_maintenance_user: A LOT of different users. Map to numbers?
2. retail_last_maintenance_user: Same as above
3. special_fee_code: numbers, characters, blanks
4. resp_specimen plan: numbers (like ids) and blanks

In [11]:
df.head()

,is_inventory_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,plan_effective_date,plan_end_date,is_resp,...,is_family_resp,is_hrdc_resp,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,0,1,0.811210,T80,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.835465,0,0.000000,NaN,0,1
1,0,1,0.929165,BATCH,0,NaN,0000,0.826910,0.000000,0,...,0,0,NaN,0.0,0.860734,0,0.954733,BATCH,1,1
2,0,1,0.726209,GUERINO,0,NaN,00/0,0.480036,0.691887,0,...,0,0,NaN,0.0,0.860734,0,0.633734,GUERINO,0,1
3,0,1,0.704650,T80,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.835465,0,0.000000,NaN,1,1
4,0,1,0.983061,H01,0,NaN,NaN,0.000000,0.000000,0,...,0,0,NaN,0.0,0.836450,0,0.000000,NaN,0,1
